In [1]:
import numpy as np
import pandas as pd
from msmarco.index import indexed
from msmarco.evaluate import grade_results, judge_queries
from time import perf_counter
from searcharray.similarity import bm25_similarity, classic_similarity, default_bm25
from searcharray import SetOfResults
# 50 QPS: 0.4783807889536503
NUM_QUERIES = 1000

/var/folders/c3/6754bm4d6cv9n0k3v04lx53h0000gq/T/ipykernel_13262/1838799328.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Will take a min or two to load, and a good hour to build an index first time
msmarco = indexed()

In [3]:
msmarco.memory_usage()

Index                 132
msmarco_id       25710680
url              25710680
title            25710680
body_idx      44433766928
title_idx       325609656
dtype: int64

In [4]:
import math

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])
    
def term_hist():
    print("Num Terms: ", len(msmarco['body_idx'].array.posns.encoded_term_posns))
    hist = []
    for i in range(0, 1000):
        term = msmarco['body_idx'].array.term_dict.get_term(i)
        docfreq = msmarco['body_idx'].array.docfreq(term)
        roaring_len = len(msmarco['body_idx'].array.posns.encoded_term_posns[i])
        hist.append( (i, term, docfreq, roaring_len) )
    
    hist.sort(key=lambda row: row[-1], reverse=True)
    
    for i in range(0,100):
        ratio = hist[i][3] / hist[i][2]
        print(f"{hist[i][0]} -- term:{hist[i][1]} | docfreq:{hist[i][2]} | bytes:{convert_size(8 * hist[i][3])} | ratio:{ratio}")

In [5]:
np.sort(msmarco['body_idx'].array.score('cheese'))

array([0.       , 0.       , 0.       , ..., 4.7565475, 5.0600863,
       8.568701 ], dtype=float32)

In [6]:
from searcharray.solr import edismax

edismax(msmarco, q='cheese curds',
        qf=['title_idx', 'body_idx'],
        mm=2,
        tie=0.2,
        pf='title_idx')

(array([0., 0., 0., ..., 0., 0., 0.]),
 '((title_idx:chees^1 | body_idx:chees^1) (title_idx:curd^1 | body_idx:curd^1))~2 (title_idx:"chees curd")^1')

In [7]:
# %%prun

def search(corpus, fields, query, query_id=None,
           pf2_fields=None,
           pf_fields=None,
           tie=0,
           mm=1,
           n=10, 
           similarities=default_bm25):
    if not isinstance(fields, list):
        fields = [fields]

    start = perf_counter()
    scored, explain = edismax(msmarco,
                              q=query,
                              qf=fields,
                              tie=tie,
                              pf=pf_fields)
                              # pf2=pf2_fields)
                              # mm=mm,
                              # similarity=similarities)
    # print(f"q={query} | qf={fields} | tie {tie} | pf={pf_fields} | pf2={pf2_fields} | mm={mm}")
    # print(explain)
    took = perf_counter() - start

    metadata = {
        'took': took,
        'query_id': query_id
    }
    return scored, n, query, metadata

search(corpus=msmarco, fields=["title_idx^10", "body_idx"], query="Adjustable-rate mortgage", n=100)[0]
# results.get_all()

array([ 1.02253187,  0.        ,  0.        , ...,  0.        ,
        0.        , 21.14721298])

In [9]:
from msmarco.evaluate import grade_results, judgments, msmarco_of_ids

msmarco_judgments = judgments().sample(NUM_QUERIES)
# ids = set(msmarco_judgments['msmarco_id'].tolist())
# msmarco_for_judgments = msmarco_of_ids(ids)
# msmarco_judgments = msmarco_judgments.merge(pd.DataFrame(msmarco_for_judgments), how='left', on='msmarco_id')
#print(msmarco_judgments.columns)
#msmarco_of_ids
# msmarco_judgments

In [10]:
import random

def random_probes(features, num_probes=10, feature_ranges={}):
    
    probes = []
    for _ in range(num_probes):
        param_dict = {}
        for param in features:
            if param in feature_ranges:
                param_dict[param] = random.uniform(feature_ranges[param][0],
                                                   feature_ranges[param][1])
            else:
                param_dict[param] = random.random() * 10
        probes.append(param_dict)
    return pd.DataFrame(probes)

probes = random_probes(features=['title_boost', 'body_boost',
                                 'title_pf_boost', 'body_pf_boost',
                                 'title_pf2_boost', 'body_pf2_boost',
                                 'tie', 'mm',
                                 'k1_title', 'b_title',
                                 'k1_body', 'b_body'],
                       feature_ranges={'k1_title': (0.1,3.0),
                                       'tie': (0,1),
                                       'mm': (0,100)})
probes

,title_boost,body_boost,title_pf_boost,body_pf_boost,title_pf2_boost,body_pf2_boost,tie,mm,k1_title,b_title,k1_body,b_body
0,0.144830,6.580237,8.543714,9.398618,6.684248,1.466168,0.224425,56.060055,1.500000,2.531442,2.562167,5.882268
1,1.903217,6.245112,9.907438,7.306416,7.701440,7.066172,0.384687,61.366408,1.921138,8.721453,5.344916,8.557910
2,5.714221,7.964295,9.060951,2.481385,3.962330,2.309485,0.664829,73.887717,1.231645,3.540308,2.780652,4.376089
3,6.898372,7.293533,6.999836,6.145602,8.310750,5.251827,0.077337,10.787573,1.717216,0.745594,6.727299,4.494990
4,6.194621,2.798459,1.545713,7.048738,3.295889,6.289102,0.920098,12.364705,2.845301,4.444710,8.039117,0.415203
5,8.090880,6.193140,8.741100,7.103978,3.517253,2.178450,0.032010,27.064394,2.115338,3.555533,0.898519,5.865027
6,0.173212,6.428415,2.731204,1.304548,8.008149,1.700908,0.453777,70.412339,1.011466,7.710929,9.565844,7.496094
7,6.381746,2.385828,9.467746,8.308390,3.936380,9.238335,0.072889,15.824271,1.190511,8.813057,3.117686,2.190659
8,7.206995,0.508757,1.442996,8.696361,8.885449,6.090403,0.570837,65.176586,1.332333,2.006784,4.023760,4.023903
9,2.607193,8.721340,4.169124,9.543435,3.365047,6.892572,0.586623,67.072434,0.727796,8.118343,6.545738,1.627235


In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed

executor = ThreadPoolExecutor()

def run_all(corpus, judgments, fields, pf_fields, pf2_fields,
            tie=0, mm=1,
            max_posns=None, min_posns=None, similarities=default_bm25, n=10):
    results = []
    query_no = 0
    results = SetOfResults(msmarco)
    futures = []
    for idx, row in judgments.iterrows():
        futures.append(executor.submit(search,
                                       corpus=corpus,
                                       fields=fields,
                                       pf_fields=pf_fields,
                                       pf2_fields=pf2_fields,
                                       similarities=similarities,
                                       n=n,
                                       query=row['query'],
                                       query_id=row['query_id']))
    start = perf_counter()
    for future in as_completed(futures):
        scored, n, query, metadata = future.result()
        results.ins_top_n(scored, N=n, query=query, metadata=metadata)
        if query_no > 0 and query_no % 50 == 0:
            print(f"-- {query_no} QPS: {query_no / (perf_counter() - start)} q:{query}")
        query_no += 1
    results = results.get_all()
    graded = grade_results(msmarco_judgments, results)
    print("Random search complete")
    return graded

def run_with_params(params,
                    msmarco,
                    msmarco_judgments,
                    n=100):
    bm25_similarity_title = bm25_similarity(b=params['b_title'],
                                            k1=params['k1_title'])
    bm25_similarity_body = bm25_similarity(b=params['b_body'],
                                           k1=params['k1_body'])
    fields = [f"title_idx^{params['title_boost']}", f"body_idx^{params['body_boost']}"]
    pf_fields = [f"title_idx^{params['title_pf_boost']}", f"body_idx^{params['body_pf_boost']}"]
    pf2_fields = [f"title_idx^{params['title_pf2_boost']}", f"body_idx^{params['body_pf2_boost']}"]
    mm=f"{int(params['mm'])}%"

    similarities = {"title_idx": bm25_similarity_title,
                    "body_idx": bm25_similarity_body}

    results = run_all(msmarco,
                      msmarco_judgments,
                      fields=fields,
                      pf2_fields=pf2_fields,
                      pf_fields=pf_fields,
                      mm=mm,
                      tie=params['tie'],
                      n=100,
                      similarities=similarities)
    return results, params


## Random search param

Seed the search with a random search

In [17]:
from random import uniform

def random_search(msmarco, msmarco_judgments, 
                  features, feature_ranges={}, times=1):
    
    results = []
    futures = []
    all_graded = None

    params = random_probes(features, feature_ranges=feature_ranges, num_probes=times)
    print(params)

    print(f"Random search over {len(params)}")
    for _, row in params.iterrows():
        graded, params = run_with_params(row, msmarco, msmarco_judgments,
                                         n=100)
        if all_graded is None:
            all_graded = graded.copy().groupby(['query'])['reciprical_rank'].first()
        else:
            all_graded += graded.copy().groupby(['query'])['reciprical_rank'].first()
        queries_judged = judge_queries(graded)
        mrr = queries_judged.sum() / len(msmarco_judgments)

        result_dict = {param_name: params[param_name] for param_name in features} 
        result_dict['mrr100'] = mrr
        results.append(result_dict)

    return pd.DataFrame(results), all_graded

initial_probes, query_perfs = random_search(msmarco, msmarco_judgments.sample(50),
                                            features=['title_boost', 'body_boost',
                                                       'title_pf_boost', 'body_pf_boost',
                                                       'title_pf2_boost', 'body_pf2_boost',
                                                       'mm', 'tie',
                                                       'k1_title', 'b_title',
                                                       'k1_body', 'b_body'])

   title_boost  body_boost  title_pf_boost  body_pf_boost  title_pf2_boost  \
0     2.247152    5.852672        4.553634       6.562009          1.91965   

   body_pf2_boost        mm       tie  k1_title   b_title  k1_body   b_body  
0        9.226755  8.708589  3.373431  4.944802  6.760062  0.12533  5.26955  
Random search over 1
Random search complete


In [18]:
query_perfs.sort_values()

query
cast of grace and frankie                                      0.0
what happens low mean platelet volume                          0.0
what is a doghouse                                             0.0
what is a logistics company                                    0.0
what is a personal assistant job description                   0.0
what is a retarder                                             0.0
what is inulin?                                                0.0
what is reversible myocardial ischemia                         0.0
what is stout mean                                             0.0
what is the routing number for td bank ma                      0.0
what is wcm server                                             0.0
what nationality is the name graves                            0.0
what network is bob's burgers on                               0.0
what team's theme is sweet caroline                            0.0
what temperature should chicken be smoked               

In [19]:
from sklearn.gaussian_process import GaussianProcessRegressor
import pandas as pd

y_train = initial_probes['mrr100']
x_train = initial_probes.drop('mrr100', axis=1)


gpr = GaussianProcessRegressor()
gpr.fit(x_train.to_numpy(), y_train.to_numpy())

GaussianProcessRegressor()

In [20]:
x_train

,title_boost,body_boost,title_pf_boost,body_pf_boost,title_pf2_boost,body_pf2_boost,mm,tie,k1_title,b_title,k1_body,b_body
0,2.247152,5.852672,4.553634,6.562009,1.91965,9.226755,8.708589,3.373431,4.944802,6.760062,0.12533,5.26955


### Score probes

In [21]:
probes

,title_boost,body_boost,title_pf_boost,body_pf_boost,title_pf2_boost,body_pf2_boost,tie,mm,k1_title,b_title,k1_body,b_body
0,0.144830,6.580237,8.543714,9.398618,6.684248,1.466168,0.224425,56.060055,1.500000,2.531442,2.562167,5.882268
1,1.903217,6.245112,9.907438,7.306416,7.701440,7.066172,0.384687,61.366408,1.921138,8.721453,5.344916,8.557910
2,5.714221,7.964295,9.060951,2.481385,3.962330,2.309485,0.664829,73.887717,1.231645,3.540308,2.780652,4.376089
3,6.898372,7.293533,6.999836,6.145602,8.310750,5.251827,0.077337,10.787573,1.717216,0.745594,6.727299,4.494990
4,6.194621,2.798459,1.545713,7.048738,3.295889,6.289102,0.920098,12.364705,2.845301,4.444710,8.039117,0.415203
5,8.090880,6.193140,8.741100,7.103978,3.517253,2.178450,0.032010,27.064394,2.115338,3.555533,0.898519,5.865027
6,0.173212,6.428415,2.731204,1.304548,8.008149,1.700908,0.453777,70.412339,1.011466,7.710929,9.565844,7.496094
7,6.381746,2.385828,9.467746,8.308390,3.936380,9.238335,0.072889,15.824271,1.190511,8.813057,3.117686,2.190659
8,7.206995,0.508757,1.442996,8.696361,8.885449,6.090403,0.570837,65.176586,1.332333,2.006784,4.023760,4.023903
9,2.607193,8.721340,4.169124,9.543435,3.365047,6.892572,0.586623,67.072434,0.727796,8.118343,6.545738,1.627235


In [22]:
from scipy.stats import norm

def score_probes(gpr, probes, best_mrr, theta):
    predictions, std_devs = gpr.predict(probes, return_std=True)
    
    probes["prediction"] = predictions
    probes["std_dev"] = std_devs
        
    probes['opportunity'] = probes['prediction'] - best_mrr - theta
    probes['prob_of_improvement'] = norm.cdf( probes['opportunity'] / probes['std_dev'])
    
    return probes.sort_values('prob_of_improvement', ascending=False)

score_probes(gpr, probes.copy(), best_mrr=1.0, theta=0.9)

/Users/douglas.turnbull/ws/msmarco/venv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(


,title_boost,body_boost,title_pf_boost,body_pf_boost,title_pf2_boost,body_pf2_boost,tie,mm,k1_title,b_title,k1_body,b_body,prediction,std_dev,opportunity,prob_of_improvement
0,0.144830,6.580237,8.543714,9.398618,6.684248,1.466168,0.224425,56.060055,1.500000,2.531442,2.562167,5.882268,0.000000e+00,1.0,-1.9,0.028717
1,1.903217,6.245112,9.907438,7.306416,7.701440,7.066172,0.384687,61.366408,1.921138,8.721453,5.344916,8.557910,0.000000e+00,1.0,-1.9,0.028717
2,5.714221,7.964295,9.060951,2.481385,3.962330,2.309485,0.664829,73.887717,1.231645,3.540308,2.780652,4.376089,0.000000e+00,1.0,-1.9,0.028717
3,6.898372,7.293533,6.999836,6.145602,8.310750,5.251827,0.077337,10.787573,1.717216,0.745594,6.727299,4.494990,2.628025e-68,1.0,-1.9,0.028717
4,6.194621,2.798459,1.545713,7.048738,3.295889,6.289102,0.920098,12.364705,2.845301,4.444710,8.039117,0.415203,5.700236e-63,1.0,-1.9,0.028717
5,8.090880,6.193140,8.741100,7.103978,3.517253,2.178450,0.032010,27.064394,2.115338,3.555533,0.898519,5.865027,9.517250e-167,1.0,-1.9,0.028717
6,0.173212,6.428415,2.731204,1.304548,8.008149,1.700908,0.453777,70.412339,1.011466,7.710929,9.565844,7.496094,0.000000e+00,1.0,-1.9,0.028717
7,6.381746,2.385828,9.467746,8.308390,3.936380,9.238335,0.072889,15.824271,1.190511,8.813057,3.117686,2.190659,1.215384e-72,1.0,-1.9,0.028717
8,7.206995,0.508757,1.442996,8.696361,8.885449,6.090403,0.570837,65.176586,1.332333,2.006784,4.023760,4.023903,0.000000e+00,1.0,-1.9,0.028717
9,2.607193,8.721340,4.169124,9.543435,3.365047,6.892572,0.586623,67.072434,0.727796,8.118343,6.545738,1.627235,0.000000e+00,1.0,-1.9,0.028717


### Bayesian search loop

In [23]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def search_w_prob_params(probes, msmarco_judgments):
    results = []
    query_perfs = None
    for idx, row in probes.iterrows():
        graded, params = run_with_params(row, msmarco,
                                         msmarco_judgments, 100)
        if query_perfs is None:
            query_perfs = graded.copy().groupby(['query'])['reciprical_rank'].first()
        else:
            query_perfs += graded.copy().groupby(['query'])['reciprical_rank'].first()

        queries_judged = judge_queries(graded)
        mrr = queries_judged.sum() / len(msmarco_judgments)

        result_dict = {'mrr100': mrr}
        for param, value in params.items():
            result_dict[param] = value
        results.append(result_dict)
        print(result_dict['mrr100'], result_dict)
    return pd.DataFrame(results), query_perfs


In [ ]:
SEED=100
NUM_QUERIES=1000
FEATURES = ['title_boost', 'body_boost',
            'title_pf_boost', 'body_pf_boost',
            'title_pf2_boost', 'body_pf2_boost',
            'tie', 'mm',
            'k1_title', 'b_title',
            'k1_body', 'b_body']
PICKLE_PATH = f"~/.msmarco/msmarco_bayes_probes_{len(FEATURES)}_{NUM_QUERIES}_{SEED}.pkl"

def bayesian_search(initial_probes, 
                    msmarco_judgments,
                    feature_ranges,
                    theta=1.2,
                    rounds=10,
                    num_probes=4):
    probes = initial_probes.copy()
    initial_cols = initial_probes.columns
    features = initial_probes.drop('mrr100', axis=1).columns
    print(features)
    query_perfs = None
    for _ in range(rounds):
        y_train = probes['mrr100']
        x_train = probes.drop('mrr100', axis=1)
        x_train = probes[features]
        
        gpr = GaussianProcessRegressor()
        gpr.fit(x_train.to_numpy(), y_train.to_numpy())

        new_probes = random_probes(x_train.columns,
                                   num_probes=100, 
                                   feature_ranges=feature_ranges)
        new_probes = score_probes(gpr, new_probes,
                                  best_mrr=probes['mrr100'].max(),
                                  theta=theta)
        new_probes = new_probes[:num_probes]
        print(new_probes.sort_values('prob_of_improvement', ascending=False))
        new_probes, round_query_perfs = search_w_prob_params(new_probes, msmarco_judgments)

        if query_perfs is None:
            query_perfs = round_query_perfs
        else:
            query_perfs += round_query_perfs

        # Concat, dropping scoring, etc
        probes = pd.concat([new_probes, probes])
        probes = probes[initial_cols]
        print(len(probes))
        print("CURRENT RESULTS")
        print(probes.sort_values('mrr100', ascending=False))
        print("WORST QUERIES")
        print(query_perfs.sort_values(ascending=False).head(10))
        print("BEST QUERIES")
        print(query_perfs.sort_values(ascending=True).head(10))
        probes.to_pickle(PICKLE_PATH)
    return probes

probes = None
try:
    probes = pd.read_pickle(PICKLE_PATH)
    print("Loaded ", PICKLE_PATH)
    print(probes)
except FileNotFoundError:
    print(f"No probes file found at {PICKLE_PATH}, starting with random search")
    pass

# Run with 1000 queries
msmarco_judgments = judgments().sample(NUM_QUERIES, random_state=SEED)
print("Random search")
feature_ranges={'k1_title': (0.1,3.0),
            'b_title':  (0.1,3.0),
            'k1_body': (0.1,3.0),
            'b_body':  (0.1,3.0),
            'tie': (0,1.0),
            'mm': (25,100)
            }
initial_probes, query_perfs = random_search(msmarco,
                                            msmarco_judgments,
                                            features=['title_boost', 'body_boost',
                                                      'title_pf_boost', 'body_pf_boost',
                                                      'title_pf2_boost', 'body_pf2_boost',
                                                      'tie', 'mm',
                                                      'k1_title', 'b_title',
                                                      'k1_body', 'b_body'],
                                            feature_ranges=feature_ranges,
                                            times=5)

print(query_perfs)
print(initial_probes)

if probes is not None:
    probes = pd.concat([probes, initial_probes])
else:
    probes = initial_probes

results = bayesian_search(probes,
                          msmarco_judgments,
                          feature_ranges=feature_ranges)
results.sort_values('mrr100', ascending=False)

No probes file found at ~/.msmarco/msmarco_bayes_probes_12_1000_100.pkl, starting with random search
Random search
   title_boost  body_boost  title_pf_boost  body_pf_boost  title_pf2_boost  \
0     2.462362    8.775352        7.193315       2.148689         0.429236   
1     9.084801    6.177775        5.695773       0.557343         0.119031   
2     9.648127    1.927283        3.356690       8.283395         6.904179   
3     4.431378    2.082981        4.586523       1.744414         4.911872   
4     8.814608    6.350059        8.092083       9.187220         6.603446   

   body_pf2_boost       tie         mm  k1_title   b_title   k1_body    b_body  
0        4.038839  0.569924  81.503925  2.622682  0.480494  0.557451  0.654765  
1        6.363155  0.757364  80.996183  1.086941  1.058246  1.580639  2.684080  
2        5.066393  0.922657  63.651684  0.717177  0.656514  1.609360  2.678514  
3        2.578916  0.830334  86.209159  1.674380  1.924115  2.265830  0.911157  
4        3.

/Users/douglas.turnbull/ws/msmarco/venv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(


-- 50 QPS: 3.2399416989412964 q:initial hiv symptoms
-- 100 QPS: 3.149342560372667 q:quarter hours equal semester hours
-- 150 QPS: 3.3405647280353743 q:most common language in india
-- 200 QPS: 3.397463813102319 q:what is esoteric
-- 250 QPS: 3.3154251729633244 q:what are argentina's cowboys called?
-- 300 QPS: 3.2438025010959843 q:what does it mean for a result to be statistically significant?
-- 350 QPS: 3.241784784305303 q:is nugenix harmful
-- 400 QPS: 3.2642858132928563 q:why is the holocaust unique
-- 450 QPS: 3.28286447571143 q:how to find mean of grouped data
-- 500 QPS: 3.3097874666811986 q:how much does it cost for a license plate
-- 550 QPS: 3.294827296600765 q:what is a layperson
-- 600 QPS: 3.2675126263177177 q:what are buprenorphine
-- 650 QPS: 3.289749952733438 q:when an atom gains or loses a valence electron it becomes a charged particle called
-- 700 QPS: 3.289562116312458 q:what to bake cupcake in
-- 750 QPS: 3.2084684006956667 q:what does the term buttercup mean
-- 

/Users/douglas.turnbull/ws/msmarco/venv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(


-- 50 QPS: 3.184412707400696 q:define pop culture
-- 100 QPS: 3.089873843422982 q:quarter hours equal semester hours
-- 150 QPS: 3.27165168751632 q:most common language in india
-- 200 QPS: 3.3580604605057593 q:what is rras
-- 250 QPS: 3.2782450228376647 q:what are argentina's cowboys called?
-- 300 QPS: 3.231441790748525 q:what does it mean for a result to be statistically significant?
-- 350 QPS: 3.223665899098221 q:is nugenix harmful
-- 400 QPS: 3.2511082507343136 q:what equipment do you need to stream netflix
-- 450 QPS: 3.2676788495218805 q:how to find mean of grouped data
-- 500 QPS: 3.3059761310713265 q:how much does it cost for a license plate
-- 550 QPS: 3.2835848667200715 q:which dynasty started the great wall of china
-- 600 QPS: 3.2630119682914414 q:what are buprenorphine
-- 650 QPS: 3.2699150559137613 q:what does ftd stand for flowers
-- 700 QPS: 3.2704891574326886 q:types candy coated licorice
-- 750 QPS: 3.2004101736757296 q:who is jimi hendrix sitting on at the pop fest

...

-- 50 QPS: 3.2807235143594577 q:what nationality is the name d'elia
-- 100 QPS: 3.118428050873646 q:quarter hours equal semester hours
-- 150 QPS: 3.3055038981062017 q:most common language in india
-- 200 QPS: 3.357991970835595 q:how much water is needed to shower in space
-- 250 QPS: 3.2870295884911274 q:what iron-containing compound carries oxygen in the bloodstream
-- 300 QPS: 3.227537245912808 q:meaning of name olav
-- 350 QPS: 3.209167806319599 q:is nugenix harmful
-- 400 QPS: 3.243902773592089 q:how long should u wait to roll the window down after tenting
-- 450 QPS: 3.2697274295748313 q:is quartz a silicate or oxide
-- 500 QPS: 3.2836864173196183 q:how much does it cost for a license plate
-- 550 QPS: 3.2774597181551184 q:what is a substitute for cream of tartar
-- 600 QPS: 3.2429718846433797 q:who is thomas nast?
-- 650 QPS: 3.2610079106095995 q:what does ftd stand for flowers
-- 700 QPS: 3.263015252980809 q:types candy coated licorice
-- 750 QPS: 3.185737652448324 q:what was t

/Users/douglas.turnbull/ws/msmarco/venv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(


-- 50 QPS: 3.226285182173453 q:what nationality is the name d'elia
-- 100 QPS: 3.1326262031637704 q:quarter hours equal semester hours
-- 150 QPS: 3.2968498465337652 q:how many seats one candidate can contest
-- 200 QPS: 3.35556476316702 q:what is esoteric
-- 250 QPS: 3.2745264154371636 q:what are argentina's cowboys called?
-- 300 QPS: 3.2323641335316067 q:meaning of name olav
-- 350 QPS: 3.2082710106644945 q:is rheumatoid arthritis same as arthritis
-- 400 QPS: 3.2388815661876835 q:how long should u wait to roll the window down after tenting
-- 450 QPS: 3.25413128607335 q:how to find mean of grouped data
-- 500 QPS: 3.287117794819773 q:how much does it cost for a license plate
-- 550 QPS: 3.2690243855933625 q:what is a substitute for cream of tartar
-- 600 QPS: 3.2493927248417234 q:what is a superscript
-- 650 QPS: 3.2581882152633597 q:when an atom gains or loses a valence electron it becomes a charged particle called
-- 700 QPS: 3.2606805046125507 q:who invented the first steam engi

/Users/douglas.turnbull/ws/msmarco/venv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(


-- 50 QPS: 3.226407552152541 q:what nationality is the name d'elia
-- 100 QPS: 3.1218681971402686 q:quarter hours equal semester hours
-- 150 QPS: 3.2609627034869217 q:allsports physiotherapy cost of appointment
-- 200 QPS: 3.3461804564120374 q:what is esoteric
-- 250 QPS: 3.24946774977434 q:what iron-containing compound carries oxygen in the bloodstream
-- 300 QPS: 3.220574543146691 q:what does it mean for a result to be statistically significant?
-- 350 QPS: 3.204348936282106 q:what makes infection around teeth
-- 400 QPS: 3.23210391175507 q:why is the holocaust unique
-- 450 QPS: 3.2551608975571615 q:how to find mean of grouped data
-- 500 QPS: 3.272820121139036 q:what makes a guinness world record
-- 550 QPS: 3.219304906428084 q:which vitamins are leached out when the foods they contain are cooked in liquid
-- 600 QPS: 3.160900376703354 q:biggest fish market in the world
-- 650 QPS: 3.1666184379512194 q:what does ftd stand for flowers
-- 700 QPS: 3.1578029723980165 q:how does the i